In [1]:
# IMPORTS
import numpy as np
import pandas as pd
from itertools import combinations
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv("advertising.csv")
X_all = df.drop(columns="Sales")
y = df["Sales"]

In [3]:
# --- 1. SINGLE VALIDATION SPLIT ---
# Train/validation split (no test set for now)
X_train, X_val, y_train, y_val = train_test_split(X_all, y, test_size=0.2, random_state=0)

# Example: try Ridge with different alpha values (hyperparameter)
alphas = [0.01, 0.1, 1, 10, 100]
single_split_results = {}

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

for alpha in alphas:
    model = Ridge(alpha=alpha)
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_val_scaled)
    mse = mean_squared_error(y_val, y_pred)
    single_split_results[alpha] = mse

print("\n--- Single validation split ---")
for alpha, mse in single_split_results.items():
    print(f"Alpha={alpha}: MSE={mse:.3f}")



--- Single validation split ---
Alpha=0.01: MSE=4.402
Alpha=0.1: MSE=4.402
Alpha=1: MSE=4.407
Alpha=10: MSE=4.535
Alpha=100: MSE=8.658


In [4]:
#  --- 2. CROSS-VALIDATION ---
cv_results = {}
X_scaled = StandardScaler().fit_transform(X_all)

for alpha in alphas:
    model = Ridge(alpha=alpha)
    scores = cross_val_score(model, X_scaled, y, scoring="neg_mean_squared_error", cv=5)
    mse_mean = -scores.mean()
    cv_results[alpha] = mse_mean

print("\n--- Cross-validation ---")
for alpha, mse in cv_results.items():
    print(f"Alpha={alpha}: CV MSE={mse:.3f}")


--- Cross-validation ---
Alpha=0.01: CV MSE=3.073
Alpha=0.1: CV MSE=3.073
Alpha=1: CV MSE=3.073
Alpha=10: CV MSE=3.151
Alpha=100: CV MSE=6.442


In [5]:
# --- 3. EXHAUSTIVE SEARCH FOR FEATURE COMBINATIONS ---
features = list(X_all.columns)
best_combo = None
best_mse = np.inf

for r in range(1, len(features) + 1):
    for combo in combinations(features, r):
        X_combo = X_all[list(combo)]
        scores = cross_val_score(Ridge(alpha=1), X_combo, y,
                                 scoring="neg_mean_squared_error", cv=5)
        mse_mean = -scores.mean()
        if mse_mean < best_mse:
            best_mse = mse_mean
            best_combo = combo

print("\n--- Exhaustive search result ---")
print(f"Best features: {best_combo} with CV MSE={best_mse:.3f}")



--- Exhaustive search result ---
Best features: ('TV', 'Radio') with CV MSE=3.012


In [6]:
# --- 4. GREEDY FORWARD SELECTION ---
remaining_features = set(features)
selected_features = []
best_mse_greedy = np.inf

while remaining_features:
    candidate_mse = {}
    for feat in remaining_features:
        trial_feats = selected_features + [feat]
        X_trial = X_all[trial_feats]
        scores = cross_val_score(Ridge(alpha=1), X_trial, y,
                                 scoring="neg_mean_squared_error", cv=5)
        mse_mean = -scores.mean()
        candidate_mse[feat] = mse_mean
    
    best_feat = min(candidate_mse, key=candidate_mse.get)
    if candidate_mse[best_feat] < best_mse_greedy:
        selected_features.append(best_feat)
        remaining_features.remove(best_feat)
        best_mse_greedy = candidate_mse[best_feat]
    else:
        break  # no improvement, stop

print("\n--- Greedy forward selection result ---")
print(f"Selected features: {selected_features} with CV MSE={best_mse_greedy:.3f}")


--- Greedy forward selection result ---
Selected features: ['TV', 'Radio'] with CV MSE=3.012
